# Biostat 257 Homework 3 - Solutions

John Baierl

**Due May 13 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.2
Commit bf53498635 (2022-02-06 15:21 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.2.0)
  CPU: Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cyclone)


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Hint:** For non-statisticians, feel free to ask for help in class or office hours. Point of this exercise is computing not statistics.

#### Solution:

From the linear mixed effects model, we know that $Y_i \sim \textrm{MVN}(X_i \beta, Z_i \Sigma Z^T_im + \sigma^2 I_n)$.  This leads to a log likelihood of:

$\begin{align}
f(Y_i | \beta, \sigma^2, \Sigma) &= \textrm{det}(2 \pi [Z_i \Sigma Z^T_i + \sigma^2 I_n])^{-\frac{1}{2}} \textrm{exp}[-\frac{1}{2}(Y_i - X_i \beta)^T (Z_i \Sigma Z^T_i + \sigma^2 I_n)^{-1}(Y_i - X_i \beta)] \\
&= (2 \pi)^{-\frac{n}{2}} \textrm{det}([Z_i \Sigma Z^T_i + \sigma^2 I_n])^{-\frac{1}{2}} \textrm{exp}[-\frac{1}{2}(Y_i - X_i \beta)^T (Z_i \Sigma Z^T_i + \sigma^2 I_n)^{-1}(Y_i - X_i \beta)] \\
\ell(\beta, \sigma^2, \Sigma|Y_i) &= -\frac{n}{2} log(2 \pi) - \frac{1}{2} log(|Z_i \Sigma Z^T_i + \sigma^2 I_n|) - \frac{1}{2}(Y_i - X^T_i \beta)^T(Z_i \Sigma Z^T_i + \sigma^2 I_n)^{-1}(Y_i - X_i\beta)
\end{align}$

Rewriting the log-liklihood to optimize computation using the Woodbury formula and determinant identity, this becomes:

$\begin{align}
\ell(\beta, \sigma^2, \Sigma|Y_i) &= -\frac{n}{2} log(2 \pi) - \frac{1}{2} log[(\sigma^2)^n det(I_q + \frac{1}{\sigma^2}L^T Z^T_i Z_i L)] - \frac{1}{2} (Y_i - X_i \beta)^T \left[\frac{1}{\sigma^2} I_n - \frac{1}{\sigma^4}(Z_i L)(I_q + \frac{1}{\sigma^2}L^T Z^T_i Z_i L)^{-1}(Z_i L)^T \right] (Y_i - X_i \beta)\\
&= -\frac{n}{2} log(2 \pi) - \frac{n}{2} log(\sigma^2) - \frac{1}{2} log[det(I_q + \frac{1}{\sigma^2}L^T Z^T_i Z_i L)] - \frac{1}{2} \left[ \frac{1}{\sigma^2} (y^T y - 2y^T X \beta + \beta^T X^T X \beta) - \frac{1}{\sigma^4} \left((L^T[Z^T_i Y - Z^T_i X \beta])^T \left[I_q + \frac{1}{\sigma^2} L^T Z^T_i Z_i L \right]^{-1} L^T(Z^T_i Y - Z^T_i X \beta) \right) \right] \\
\end{align}$

Well that certainly is a mouthfull!  Let's go ahead and code it up.

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [1]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p      :: Vector{T}
    storage_p2     :: Vector{T}
    storage_q      :: Vector{T}
    storage_q2     :: Vector{T}
    ynorm          :: T
    xtx            :: Matrix{T}
    xty            :: Vector{T}
    ztx            :: Matrix{T}
    zty            :: Vector{T}
    ztz            :: Matrix{T}
    storage_qq     :: Matrix{T}
    storage_qq2    :: Matrix{T}
end

# constructor, makes instances of those types
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p      = Vector{T}(undef, size(X, 2))
    storage_p2     = Vector{T}(undef, size(X, 2))
    storage_q      = Vector{T}(undef, size(Z, 2))
    storage_q2     = Vector{T}(undef, size(Z, 2))
    ynorm          = norm(y)
    xtx            = transpose(X) * X
    xty            = transpose(X) * y
    ztx            = transpose(Z) * X
    zty            = transpose(Z) * y
    ztz            = transpose(Z) * Z
    storage_qq     = similar(ztz)
    storage_qq2    = similar(ztz)
    LmmObs(y, X, Z, storage_p, storage_p2, storage_q, storage_q2,
        ynorm, xtx, xty, ztx, zty, ztz, storage_qq, storage_qq2)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [3]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2) 
    # TODO: compute and return the log-likelihood
    sleep(1e-3) # wait 1 ms as if your code takes 1ms
    return zero(T)
end

logl! (generic function with 1 method)

**Hint**: This function shouldn't be very long. Mine, obeying 80-character rule, is 25 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm first then use BLAS functions to reduce memory allocations.

#### Solution:

**Step 1: Prototyping**

In [9]:
function logl_prototype!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    

    loglik = (-n / 2) * log(2pi) - (1 / 2) * logdet(Z * L * transpose(Z * L) + 
        Diagonal(fill(σ², n))) - (1 / 2)transpose(y - X * β) * 
        inv(Z * L * transpose(Z * L) + 
        Diagonal(fill(σ², n))) * (y - X * β)
    return loglik
end

logl_prototype! (generic function with 1 method)

**Step 2: Optimizing**

In [2]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    
    #Compute Cholesky for M = I + 1/sig2 .* L'Z'ZL
    mul!(obs.storage_qq, obs.ztz, L)
    mul!(obs.storage_qq2, transpose(L), obs.storage_qq)
    obs.storage_qq2 .= (σ²^(-1)) .* obs.storage_qq2
    for i in 1:q; obs.storage_qq2[i, i] += 1; end
    LAPACK.potrf!('L', obs.storage_qq2)
    
    #Store L'(Z'Y - Z'X*beta) in qx1 vector
    mul!(obs.storage_q2, obs.ztx, β)
    obs.storage_q2 .= obs.zty .- obs.storage_q2
    mul!(obs.storage_q, transpose(L), obs.storage_q2)
    
    #Compute r'r = (y - X*beta)'(y - X*beta) = y'y - 2y'x*beta + beta'*X'X*beta
    mul!(obs.storage_p, obs.xtx, β)
    obs.storage_p2 .= -2 .* obs.xty .+ obs.storage_p 
    rtr = obs.ynorm^2 + dot(β, obs.storage_p2)
    
    #Compute term with matrix inverse: 1/2 * (y - x*beta)'(M)^-1 (y - x*beta)
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq2, obs.storage_q)
    inv = (1 / 2) * ((σ²^(-1)) * rtr - (σ²^(-2)) * BLAS.nrm2(q, obs.storage_q, 1)^2)
    
    #Compute determinant from Cholesky
    d = zero(T)
    for i in 1:q; d += log(obs.storage_qq2[i, i]); end
    
    #Computes full log-likelihood
    loglik = (n / 2) * (-log(σ²) - log(2pi)) - d - inv
    return loglik      
end

logl! (generic function with 1 method)

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [3]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z);
obs;

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [4]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.1793358058317

Check that your answer matches that from Distributions.jl

In [5]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.17933580584

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [14]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [15]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 8151 samples with 1 evaluation.
 Range (min … max):  582.667 μs …  4.387 ms  ┊ GC (min … max): 0.00% … 84.59%
 Time  (median):     599.291 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   609.155 μs ± 72.096 μs  ┊ GC (mean ± σ):  0.19% ±  1.54%

    ▄▇██▇▅▄▄▃▃▃▂▂▁▁▁                                           ▂
  ▄████████████████████▇▇▇▆▆▅▅▅▄▅▇▆▅▅▅▅▇▅▅▅▆▅▅▅▅▅▆▄▄▆▃▅▅▅▃▃▅▃▄ █
  583 μs        Histogram: log(frequency) by time       765 μs <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [6]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 262 evaluations.
 Range (min … max):  298.504 ns … 448.317 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     305.660 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   306.773 ns ±   5.894 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▁▃▆▇█▇▇▅▃            ▁▁▃▂▁                            ▂
  ▆▇▇▇▆▄▃▄██████████████████▇▇▇█████▇▆▅▅▃▅▅▃▄▄▆▅▆▄▅▁▄▄▅▄▅▅▅▄▅▁▄ █
  299 ns        Histogram: log(frequency) by time        332 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [17]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

30.0

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [18]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

30.0

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.